# Structured Generation from Images or Documents Using Vision Language Models

In this example, we will use the `SmolVLM-Instruct` model to extract structured information from documents. We will run the VLM using the Transformers library and the [`Outlines`](https://github.com/dottxt-ai/outlines) library, which facilitates structured generation based on limiting token sampling probabilities.

## Setups

In [ ]:
!pip install -qU accelerate outlines transformers torch flash-attn datasets sentencepiece

In [ ]:
import outlines
import torch

from datasets import load_dataset
from outlines.models.transformers_vision import transformers_vision
from transformers import AutoProcessor, AutoModelForImageTextToText
from pydantic import BaseModel

## Initialize the model

We start by initializing the [`HuggingFaceTB/SmolVLM-Instruct`](https://huggingface.co/HuggingFaceTB/SmolVLM-Instruct).

Outlines expects us to pass in a model class and processor class.

In [ ]:
model_name = "HuggingFaceTB/SmolVLM-Instruct"

def get_model_and_processor_class(model_name: str):
    processor = AutoProcessor.from_pretrained(model_name)
    model = AutoModelForImageTextToText.from_pretrained(model_name)

    classes = mode.__class__, processor.__class__
    del model, processor

    return classes

In [ ]:
model_class, processor_class = get_model_and_processor_class(model_name)

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

model = transformers_vision(
    model_name,
    model_class=model_class,
    device=device,
    model_kwargs={'torch_dtype': torch.bfloat16, 'device_map': 'auto'},
    processor_class=processor_class,
    processor_kwargs={'device': device},
)

## Structured generation

We will define a function that defines how the output of our model will be structured. We will use the [`openbmb/RLAIF-V-Dataset`](https://huggingface.co/datasets/openbmb/RLAIF-V-Dataset) dataset, which contains a set of images along with questions and their chosen and rejected responses.

We want to create additional text-image-to-text data on top of the images to get our own structured dataset, and potentially finetune our model on it. We will use the model to generate a caption, a question, and a simple quality tag for the image.

In [ ]:
class ImageData(BaseModel):
    quality: str
    description: str
    question: str


structured_generator = outlines.generate.json(model, ImageData)

Next, we will define an extraction prompt:

In [ ]:
prompt = """
You are an image analysis assisant.

Provide a quality tag, a description and a question.

The quality can either be "good", "okay" or "bad".
The question should be concise and objective.

Return your response as a valid JSON object.
""".strip()

Load our dataset:

In [ ]:
dataset = load_dataset('openbmb/RLAIF-V-Dataset', split='train[:10]')
dataset

Next, we will define a function that will extract the structured information from the image. We will format the prompt using the `apply_chat_template` method and pass it to the model along with the image after that.

In [ ]:
def extract(row):
    messages = [
        {
            'role': 'user',
            'content': [
                {
                    'type': 'image'
                },
                {
                    'type': 'text',
                    'text': prompt
                }
            ]
        }
    ]

    formatted_prompt = model.processor.apply_chat_template(
        messages,
        add_generation_prompt=True
    )
    result = structured_generator(formatted_prompt, [row['image']])
    row['synthetic_question'] = result.question
    row['synthetic_description'] = result.description
    row['synthetic_quality'] = result.quality

    return row

In [ ]:
dataset = dataset.map(lambda x: extract(x))
dataset

In [ ]:
dataset[0]